In [1]:
import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import create_aggregate_df as agg_df

%matplotlib inline
pd.set_option('display.max_columns', None);

### Read in 2013 accepted loan data, 2014 accepted loan data, and 2013-2014 rejected loan data

In [2]:
#get 2013 data
lc2013 = pd.read_csv("../data/2012_13_loan_data/LoanStats3b.csv", low_memory = False, encoding='latin-1');
# lc2013.head()

In [3]:
#read in 2014 accepted loan data
lc14a = pd.read_csv("../data/2014_loan_data/LoanStats_2014_A.csv")
lc14b = pd.read_csv("../data/2014_loan_data/LoanStats_2014_B.csv")
lc14c = pd.read_csv("../data/2014_loan_data/LoanStats_2014_C.csv")
lc14d = pd.read_csv("../data/2014_loan_data/LoanStats_2014_D.csv")

/Users/bhavenpatel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
acc_loans_2014 = pd.concat([lc14a, lc14b, lc14c, lc14d], axis = 0);
print(acc_loans_2014.shape)
# acc_loans_2014.head()

(235629, 145)


In [5]:
#get 2013-2014 rejected loan data
lc1314rejecta = pd.read_csv("../data/2014_loan_data/RejectStats2013_2014_A.csv", header=1)
lc1314rejectb = pd.read_csv("../data/2014_loan_data/RejectStats2013_2014_B.csv", header=None)
lc1314rejectc = pd.read_csv("../data/2014_loan_data/RejectStats2013_2014_C.csv", header=None)
lc1314rejectd = pd.read_csv("../data/2014_loan_data/RejectStats2013_2014_D.csv", header=None)
lc1314rejectb.columns = lc1314rejecta.columns
lc1314rejectc.columns = lc1314rejecta.columns
lc1314rejectd.columns = lc1314rejecta.columns

In [6]:
# bind all 2013/2014 LC reject data together
lc_reject_all = lc1314rejecta.append(lc1314rejectb).append(lc1314rejectc).append(lc1314rejectd).reset_index()
lc_reject_all.shape

(2694642, 10)

In [8]:
lc_reject_all.head()

,index,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,0,30000.0,2013-01-01,debt_consolidation,754.0,32.01%,559xx,MN,< 1 year,0
1,1,15000.0,2013-01-01,medical,728.0,27.01%,945xx,CA,< 1 year,0
2,2,2500.0,2013-01-01,other,723.0,0.92%,190xx,PA,7 years,0
3,3,4000.0,2013-01-01,car,563.0,24.92%,296xx,SC,< 1 year,0
4,4,22000.0,2013-01-01,debt_consolidation,0.0,26.91%,735xx,OK,< 1 year,0


### Split 2013-2014 rejected loans into their respective years

In [9]:
def format_app_year(x):
    year = x[0:4];
    return int(year);

lc_reject_all['App_year'] = lc_reject_all['Application Date'].map(format_app_year);
lc_reject_all.head()

,index,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code,App_year
0,0,30000.0,2013-01-01,debt_consolidation,754.0,32.01%,559xx,MN,< 1 year,0,2013
1,1,15000.0,2013-01-01,medical,728.0,27.01%,945xx,CA,< 1 year,0,2013
2,2,2500.0,2013-01-01,other,723.0,0.92%,190xx,PA,7 years,0,2013
3,3,4000.0,2013-01-01,car,563.0,24.92%,296xx,SC,< 1 year,0,2013
4,4,22000.0,2013-01-01,debt_consolidation,0.0,26.91%,735xx,OK,< 1 year,0,2013


In [10]:
#get 2013 rejected loans
rejected_loans_2013 = lc_reject_all[lc_reject_all.App_year == 2013];
rejected_loans_2013.shape

(760942, 11)

In [12]:
#get 2014 rejected loans
rejected_loans_2014 = lc_reject_all[lc_reject_all.App_year == 2014];
print(rejected_loans_2014.shape)
assert(lc_reject_all.shape[0] == (rejected_loans_2013.shape[0]+rejected_loans_2014.shape[0]) ) #check that we have all the loans accounted for

(1933700, 11)


### Calculate 2014 accepted loans and rejected loans based on zip code

In [20]:
#format 3-digit zipcode for each accepted loan
acc_loans_2014['3_dig_zip'] = acc_loans_2014.zip_code.str.slice(0,3);
# acc_loans_2014['3_dig_zip'].unique() #check that zipcodes are formatted correctly
len(acc_loans_2014['3_dig_zip'].unique())

866

In [21]:
#format 3-digit zipcode for each rejected loan
rejected_loans_2014['3_dig_zip'] = rejected_loans_2014['Zip Code'].str.slice(0,3);
# rejected_loans_2014['3_dig_zip'].unique() #check that zipcodes are formatted correctly
len(rejected_loans_2014['3_dig_zip'].unique())

/Users/bhavenpatel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


996

In [28]:
#group accepted loans by 3 digit zip code
acc_loans_2014_zipgb = acc_loans_2014.groupby(by='3_dig_zip').agg({
    '3_dig_zip' : ['count'],
});

#group rejected loans by 3 digit zip code
rej_loans_2014_zipgb = rejected_loans_2014.groupby(by='3_dig_zip').agg({
    '3_dig_zip' : ['count'],
});
# rej_loans_2014_zipgb

In [31]:
acc_loans_2014_zipgb.columns = ['2014_accpt_loans']
rej_loans_2014_zipgb.columns = ['2014_rejected_loans']

In [41]:
rej_loans_2014_zipgb.tail(15)

,2014_rejected_loans
3_dig_zip,
985,2779
986,2918
987,15
988,1039
989,1273
990,732
991,470
992,1633
993,1643


In [48]:
loan_cnt_2014_zip3 = acc_loans_2014_zipgb.merge(rej_loans_2014_zipgb, on='3_dig_zip', how='left')

In [50]:
loan_cnt_2014_zip3['2014_Total_loans_considered'] = loan_cnt_2014_zip3['2014_accpt_loans'] + loan_cnt_2014_zip3['2014_rejected_loans'];
loan_cnt_2014_zip3['2014_pct_accepted'] = loan_cnt_2014_zip3['2014_accpt_loans'] / loan_cnt_2014_zip3['2014_Total_loans_considered'];
loan_cnt_2014_zip3.head()

,2014_accpt_loans,2014_rejected_loans,2014_Total_loans_considered,2014_pct_accepted
3_dig_zip,,,,
007,2,63.0,65.0,0.030769
008,1,45.0,46.0,0.021739
010,379,2741.0,3120.0,0.121474
011,115,1367.0,1482.0,0.077598
012,93,803.0,896.0,0.103795


In [51]:
loan_cnt_2014_zip3.to_csv('../data/2014_loan_data/pct_accepted_by_zip3_2014_BP.csv');